In [ ]:
import cv2
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import torch

from pathlib import Path
import sklearn
import skimage
from skimage import __version__ as skimage_version
from sklearn import __version__ as sklearn_version


path = "/content/"
debug = True

from google.colab import drive
drive.mount(path+'/drive/')

if debug == True:
    A5_savepath = path+'/drive/MyDrive/03 McGill EE Semester 5 (Fall 2025)/ECSE 415 (Clark)/A5/'

print(f"Python version:           {os.sys.version.split()[0]}")
print(f"OpenCV version:           {cv2.__version__}")
print(f"NumPy version:            {np.__version__}")
print(f"Matplotlib version:       {matplotlib.__version__}")
print(f"PyTorch version:          {torch.__version__}")
print(f"scikit-image version:     {skimage_version}")
print(f"scikit-learn version:     {sklearn_version}")
print("Path: " + path)


Python version:           3.11.5
OpenCV version:           4.12.0
NumPy version:            2.2.6
Matplotlib version:       3.10.6
PyTorch version:          2.9.0+cpu
scikit-image version:     0.25.2
scikit-learn version:     1.7.2
Path: /content/


In [ ]:
if debug == False:
    os.chdir(path)
    !pip install kaggle
    !mkdir -p ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c ecse-415-video-analysis
    !unzip -q ecse-415-video-analysis.zip -d .

In [ ]:

task1_dir = Path(path, "Object_Tracking", "Task1")
csv1_path = Path(task1_dir, "gt", "gt.txt")
ground_truth_task1 = np.loadtxt(csv1_path, delimiter=",")

images1_path = Path(task1_dir, "images")
first_iter = True
for img in os.listdir(images1_path):
    img_path = Path(images1_path, img)
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    if first_iter == True: 
        H, W, _ = img_rgb.shape
        first_iter == False
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
        fps = 14.0
        frame_size = (W,H)
        if debug == True:
            mp4_path = os.path.join(A5_savepath, 'task1_input.mp4') 
        else:
            mp4_path = path
        writer = cv2.VideoWriter(mp4_path, fourcc,fps,frame_size)
    writer.write(img_rgb)
writer.release()
print(f"Wrote .mp4 to {mp4_path}")


In [ ]:
# Source: https://www.kaggle.com/code/nityampareek/using-deepsort-object-tracker-with-yolov5

import cv2
import numpy as np
import sys
import glob

import time
import torch

# Creating a class for object detection which plots boxes and scores frames in addition to detecting an 
# object

class YoloDetector():

    def __init__(self):
        #Using yolov5s for our purposes of object detection, you may use a larger model
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('Using Device: ', self.device)
    
    def score_frame(self, frame):
        self.model.to(self.device)
        downscale_factor = 2
        width = int(frame.shape[1] / downscale_factor)
        height = int(frame.shape[0] / downscale_factor)
        frame = cv2.resize(frame, (width, height))

        results = self.model(frame)

        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

        return labels, cord
    
    def class_to_label(self, x):
        return self.classes[int(x)]
    
    def plot_boxes(self, results, frame, height, width, confidence=0.3):

        labels, cord = results
        detections = []

        n = len(labels)
        x_shape, y_shape = width, height

        for i in range(n):
            row = cord[i]

            if row[4]>=confidence:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                
                #In this demonstration, we will only be detecting persons. You can add classes of your choice
                if self.class_to_label(labels[i]) == 'car':
                    x_center = x1 + (x2-x1)
                    y_center = y1 + ((y2-y1) / 2)
                    tlwh = np.asarray([x1, y1, int(x2-x1), int(y2-y1)], dtype = np.float32)
                    confidence = float(row[4].item())
                    feature = 'car'
                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], row[4].item(), 'car'))
        return frame, detections

# Setting input video to webcam
# To use your own pre-downloaded videos, write the file path instead of 0
cap = cv2.VideoCapture(0)

# Setting resolution for webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

#Initializing the detection class
detector = YoloDetector()

